# The Battle of Neighbourhoods - Capstone Project 

## Best Neighbourhood Recommender system

In [1]:
# importing libraries
import numpy as np
import pandas as pd 
from bs4 import BeautifulSoup
import requests 
import json 
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#!conda install -c conda-forge geopy --yes
#import geopy.geocoders # convert an address into latitude and longitude values
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
print('Libraries are imported.')

Libraries are imported.


### Importing data prepared in the last assignment

In [2]:
df = pd.read_csv('Final_Data.csv')
df.head()

,Unnamed: 0,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [3]:
df.drop(['Unnamed: 0'], axis = 1) 

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


### Creating map of toronto

In [4]:
#latitude and longtitude are manually extracted from google
latitude = 43.6932; longitude = -79.3832
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

#add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto) 
map_toronto

### Working on Downtown toronto and its neighborhood

In [5]:
DT_data = df[df['Borough'] == 'Downtown Toronto']
DT_data = DT_data.reset_index(drop=True).drop(columns = 'Unnamed: 0')
DT_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [6]:
DT_latitude = 43.6548
DT_longitude = -79.3883
print('The geograpical coordinate of Downtown Toronto are: {}, {}.'.format(DT_latitude, DT_longitude))

map_DT = folium.Map(location=[DT_latitude, DT_longitude], zoom_start=10)
# add markers to map
for lat, lng, label in zip(DT_data['Latitude'], DT_data['Longitude'], DT_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 10,
        popup = label,
        color ='blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_DT)   
map_DT

The geograpical coordinate of Downtown Toronto are: 43.6548, -79.3883.


In [7]:
CLIENT_ID = 'I3X2DC0WECWLN0NMGCJCEEIP3JJ5VMY31TB25NDECMKCGGJQ' # your Foursquare ID
CLIENT_SECRET = 'ZE41RAWQA2MJDBTL13RC3NR0YSMJLKZX0DHXKAN32VB00MXB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: I3X2DC0WECWLN0NMGCJCEEIP3JJ5VMY31TB25NDECMKCGGJQ
CLIENT_SECRET:ZE41RAWQA2MJDBTL13RC3NR0YSMJLKZX0DHXKAN32VB00MXB


### Using foursquare api and getting venues data

In [8]:
def foursquare_crawler (postal_code, neighborhood, lat, lng, LIMIT = 500, radius = 1500):
    result_ds = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code, neighborhood, lat, lng):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code 
        tmp_dict['Neighborhood(s)'] = neighborhood
        tmp_dict['Latitude'] = lat
        tmp_dict['Longitude'] = lng
        tmp_dict['Crawling_result'] = results
        result_ds.append(tmp_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data obtained for the {} and Neighborhoods {}'.format(postal_code, neighborhood))
    return result_ds;


In [9]:
toronto_data = foursquare_crawler(list(DT_data['Postal Code']),
                                  list(DT_data['Neighborhood']),
                                   list(DT_data['Latitude']),
                                   list(DT_data['Longitude'])
                                  )

1.
Data obtained for the M5A and Neighborhoods Regent Park, Harbourfront
2.
Data obtained for the M7A and Neighborhoods Queen's Park, Ontario Provincial Government
3.
Data obtained for the M5B and Neighborhoods Garden District, Ryerson
4.
Data obtained for the M5C and Neighborhoods St. James Town
5.
Data obtained for the M5E and Neighborhoods Berczy Park
6.
Data obtained for the M5G and Neighborhoods Central Bay Street
7.
Data obtained for the M6G and Neighborhoods Christie
8.
Data obtained for the M5H and Neighborhoods Richmond, Adelaide, King
9.
Data obtained for the M5J and Neighborhoods Harbourfront East, Union Station, Toronto Islands
10.
Data obtained for the M5K and Neighborhoods Toronto Dominion Centre, Design Exchange
11.
Data obtained for the M5L and Neighborhoods Commerce Court, Victoria Hotel
12.
Data obtained for the M5S and Neighborhoods University of Toronto, Harbord
13.
Data obtained for the M5T and Neighborhoods Kensington Market, Chinatown, Grange Park
14.
Data obtain

In [10]:
toronto_data

[{'Postal Code': 'M5A',
  'Neighborhood(s)': 'Regent Park, Harbourfront',
  'Latitude': 43.6542599,
  'Longitude': -79.3606359,
  'Crawling_result': [{'reasons': {'count': 0,
     'items': [{'summary': 'This spot is popular',
       'type': 'general',
       'reasonName': 'globalInteractionReason'}]},
    'venue': {'id': '54ea41ad498e9a11e9e13308',
     'name': 'Roselle Desserts',
     'location': {'address': '362 King St E',
      'crossStreet': 'Trinity St',
      'lat': 43.653446723052674,
      'lng': -79.3620167174383,
      'labeledLatLngs': [{'label': 'display',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383}],
      'distance': 143,
      'postalCode': 'M5A 1K9',
      'cc': 'CA',
      'city': 'Toronto',
      'state': 'ON',
      'country': 'Canada',
      'formattedAddress': ['362 King St E (Trinity St)',
       'Toronto ON M5A 1K9',
       'Canada']},
     'categories': [{'id': '4bf58dd8d48988d16a941735',
       'name': 'Bakery',
       'pluralName': 'B

### Getting clean venue data and its information from neighbourhood

In [25]:
# extract each venue for respective neighborhood
def get_venue_from_dataset(data):
    result_df = pd.DataFrame(columns = ['Postal Code', 'Neighbourhood', 
                                        'Neighborhood Latitude', 
                                        'Neighborhood Longitude',
                                        'Venue', 'Venue Summary', 
                                        'Venue Category', 'Distance'])
    for neigh in data:
        postal_code = neigh['Postal Code'] 
        neighbourhood = neigh['Neighborhood(s)']
        lat = neigh['Latitude']
        lng = neigh['Longitude']
        for venue in neigh['Crawling_result']:
            summary = venue['reasons']['items'][0]['summary']
            name = venue['venue']['name']
            dist = venue['venue']['location']['distance']
            cat =  venue['venue']['categories'][0]['name']
            
            result_df = result_df.append({'Postal Code': postal_code, 'Neighbourhood': neighbourhood, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)   
    return(result_df)

In [26]:
toronto_venues = get_venue_from_dataset(toronto_data)

In [27]:
toronto_venues.head() #Showing venues and its information for each neighbourhood

,Postal Code,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M5A,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,This spot is popular,Bakery,143
1,M5A,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,This spot is popular,Coffee Shop,122
2,M5A,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,This spot is popular,Restaurant,376
3,M5A,"Regent Park, Harbourfront",43.65426,-79.360636,The Distillery Historic District,This spot is popular,Historic Site,459
4,M5A,"Regent Park, Harbourfront",43.65426,-79.360636,Distillery Sunday Market,This spot is popular,Farmers Market,475


### Getting some idea about venues in toroto

In [28]:
neigh_list = list(toronto_venues['Neighbourhood'].unique())
print('Number of Neighborhoods in downtown toronto is {}:'.format(len(neigh_list)))
print('List of Neighborhoods in downtown toronto:')
neigh_list

Number of Neighborhoods in downtown toronto is 19:
List of Neighborhoods in downtown toronto:


['Regent Park, Harbourfront',
 "Queen's Park, Ontario Provincial Government",
 'Garden District, Ryerson',
 'St. James Town',
 'Berczy Park',
 'Central Bay Street',
 'Christie',
 'Richmond, Adelaide, King',
 'Harbourfront East, Union Station, Toronto Islands',
 'Toronto Dominion Centre, Design Exchange',
 'Commerce Court, Victoria Hotel',
 'University of Toronto, Harbord',
 'Kensington Market, Chinatown, Grange Park',
 'CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport',
 'Rosedale',
 'Stn A PO Boxes',
 'St. James Town, Cabbagetown',
 'First Canadian Place, Underground city',
 'Church and Wellesley']

In [29]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
print('Different categories:')
list(toronto_venues['Venue Category'].unique())

There are 190 uniques categories.
Different categories:


['Bakery',
 'Coffee Shop',
 'Restaurant',
 'Historic Site',
 'Farmers Market',
 'Park',
 'Mediterranean Restaurant',
 'Chocolate Shop',
 'Dessert Shop',
 'Liquor Store',
 'Performing Arts Venue',
 'Tech Startup',
 'Italian Restaurant',
 'French Restaurant',
 'Thai Restaurant',
 'Pool',
 'Pub',
 'Theater',
 'Diner',
 'Gym / Fitness Center',
 'Hotel',
 'Café',
 'Middle Eastern Restaurant',
 'Grocery Store',
 'Animal Shelter',
 'Athletics & Sports',
 'Food Truck',
 'Hotel Bar',
 'Gastropub',
 'Art Gallery',
 'Brewery',
 'BBQ Joint',
 'Cosmetics Shop',
 'Japanese Restaurant',
 'Circus',
 'Creperie',
 'Bar',
 'Vegetarian / Vegan Restaurant',
 'Taco Place',
 'Curling Ice',
 'Pet Store',
 'Wine Bar',
 'Farm',
 'Dive Bar',
 'Gift Shop',
 'Electronics Store',
 'Garden',
 'Rock Club',
 'Asian Restaurant',
 'Vietnamese Restaurant',
 'Deli / Bodega',
 'Ice Cream Shop',
 'Taiwanese Restaurant',
 'Bubble Tea Shop',
 'Dance Studio',
 'Beer Bar',
 'Yoga Studio',
 'Arts & Crafts Store',
 'Distribution 

### One hot encoding venue categories

In [30]:
toronto_onehot = pd.get_dummies(data = toronto_venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
toronto_onehot.head()

,Postal Code,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,Accessories Store,Airport,American Restaurant,Animal Shelter,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Diner,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Nudist Beach,Opera House,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Tourist Information Center,Track,Trail,Train Station,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M5A,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,This spot is popular,143,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M5A,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,This spot is popular,122,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M5A,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,This spot is popular,376,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [32]:
# Creating list of restaurants manually
important_features_list = [
 'Neighbourhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude', 
 'American Restaurant',
 'Asian Restaurant', 
 'BBQ Joint', 
 'Bakery',   
 'Belgian Restaurant',
 'Bistro',
 'Bubble Tea Shop',
 'Breakfast Spot',
 'Burger Joint',
 'Comfort Food Restaurant',
 'Cuban Restaurant', 
 'Caribbean Restaurant',
 'Chinese Restaurant', 
 'Doner Restaurant',
 'Dumpling Restaurant',
 'Eastern European Restaurant',
 'Ethiopian Restaurant',
 'Falafel Restaurant',
 'Fast Food Restaurant',
 'Filipino Restaurant',
 'Fish Market',
 'Food Court',
 'Food Truck',
 'Food & Drink Shop',
 'French Restaurant',
 'Fried Chicken Joint', 
 'Greek Restaurant',
 'Grocery Store',
 'Greek Restaurant',
 'Indian Restaurant',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Jewish Restaurant',
 'Korean Restaurant',
 'Latin American Restaurant',
 'Mediterranean Restaurant', 
 'Mexican Restaurant',
 'Middle Eastern Restaurant', 
 'Modern European Restaurant',
 'New American Restaurant',
 'Persian Restaurant',
 'Pizza Place',
 'Restaurant',
 'Sandwich Place',
 'Seafood Restaurant', 
 'Sushi Restaurant',
 'Taiwanese Restaurant',
 'Thai Restaurant', 
 'Vegetarian / Vegan Restaurant',
 'Vietnamese Restaurant',
 'Wings Joint']

### Grouping data according to the neighbourhood

In [ ]:
toronto_onehot = toronto_onehot[important_features_list].drop(
    columns = ['Neighborhood Latitude', 'Neighborhood Longitude']).groupby('Neighbourhood').sum()
toronto_onehot.head()

### Counting total restaurants and joints in neighbourhood

In [35]:
feat_list = list(toronto_onehot.columns)
restaurant_list = []
for counter, value in enumerate(feat_list):
    if value.find('Restaurant') != (-1):
        restaurant_list.append(value)
        
toronto_onehot['Total Restaurants'] = toronto_onehot[restaurant_list].sum(axis = 1)
toronto_onehot = toronto_onehot.drop(columns = restaurant_list)

feat_list = list(toronto_onehot.columns)
joint_list = []

for counter, value in enumerate(feat_list):
    if value.find('Joint') != (-1):
        joint_list.append(value)
        
toronto_onehot['Total Joints'] = toronto_onehot[joint_list].sum(axis = 1)
toronto_onehot = toronto_onehot.drop(columns = joint_list)

### Final dataset

In [36]:
toronto_onehot

,Bakery,Bistro,Bubble Tea Shop,Breakfast Spot,Fish Market,Food Court,Food Truck,Food & Drink Shop,Grocery Store,Pizza Place,Sandwich Place,Total Restaurants,Total Joints
Neighbourhood,,,,,,,,,,,,,
Berczy Park,3,0,1,0,0,0,1,0,1,1,0,23,1
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",0,0,0,0,0,0,0,0,0,2,0,8,1
Central Bay Street,1,0,2,1,0,0,0,0,1,2,2,17,3
Christie,3,0,0,0,0,0,0,0,3,0,1,30,1
Church and Wellesley,0,0,2,1,0,0,0,0,3,1,1,18,1
"Commerce Court, Victoria Hotel",1,0,2,0,0,0,1,1,1,0,2,22,1
"First Canadian Place, Underground city",1,0,2,0,0,0,1,1,1,1,3,18,1
"Garden District, Ryerson",1,0,1,0,0,0,1,0,0,1,1,24,2
"Harbourfront East, Union Station, Toronto Islands",2,0,1,0,0,0,1,0,0,2,0,16,1


### Applying K-Means Clustering with 5 clusters

In [37]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(toronto_onehot)

### Looking at centers of each cluster

In [38]:
mean_df = pd.DataFrame(kmeans.cluster_centers_)
mean_df.columns = toronto_onehot.columns
mean_df.index = ['G1','G2','G3','G4','G5']
mean_df['Total Sum'] = mean_df.sum(axis = 1)
mean_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Bakery,Bistro,Bubble Tea Shop,Breakfast Spot,Fish Market,Food Court,Food Truck,Food & Drink Shop,Grocery Store,Pizza Place,Sandwich Place,Total Restaurants,Total Joints,Total Sum
G5,4.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,1.000000,0.000000,4.000000,3.000000,26.000000,0.000000,39.000000
G3,3.666667,0.333333,0.000000,0.333333,0.333333,0.0,0.0,0.000000,3.000000,0.666667,0.666667,28.333333,0.666667,38.000000
G2,2.250000,0.000000,1.000000,0.250000,0.000000,0.0,1.0,0.250000,0.875000,0.750000,0.875000,22.625000,1.250000,31.125000
G1,1.166667,0.000000,1.666667,0.666667,0.000000,0.0,0.5,0.333333,1.166667,1.333333,1.666667,17.666667,1.166667,27.333333
G4,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,2.000000,0.000000,8.000000,1.000000,11.000000


#### From the above dataframe we can conclude that best neighbourhood group is G5 and worst group is G4.

### Finding group of respective neighbourhoods

In [41]:
neigh_summary = pd.DataFrame([toronto_onehot.index, 1 + kmeans.labels_]).T
neigh_summary.columns = ['Neighborhood', 'Group']
neigh_summary

,Neighborhood,Group
0,Berczy Park,2
1,"CN Tower, King and Spadina, Railway Lands, Har...",4
2,Central Bay Street,1
3,Christie,3
4,Church and Wellesley,1
5,"Commerce Court, Victoria Hotel",2
6,"First Canadian Place, Underground city",1
7,"Garden District, Ryerson",2
8,"Harbourfront East, Union Station, Toronto Islands",1
9,"Kensington Market, Chinatown, Grange Park",5


### Best Neighbourhood Details

In [42]:
neigh_summary[neigh_summary['Group'] == 5]

,Neighborhood,Group
9,"Kensington Market, Chinatown, Grange Park",5


In [44]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 5]['Neighborhood'])[0]
toronto_venues[toronto_venues['Neighbourhood'] == name_of_neigh].iloc[0,1:4].to_dict()

{'Neighbourhood': 'Kensington Market, Chinatown, Grange Park',
 'Neighborhood Latitude': 43.6532057,
 'Neighborhood Longitude': -79.4000493}

### Second Best Neighbourhood Details

In [46]:
neigh_summary[neigh_summary['Group'] == 3]

,Neighborhood,Group
3,Christie,3
13,Rosedale,3
18,"University of Toronto, Harbord",3


In [50]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 3]['Neighborhood'])[0]
toronto_venues[toronto_venues['Neighbourhood'] == name_of_neigh].iloc[0,1:4].to_dict()

{'Neighbourhood': 'Christie',
 'Neighborhood Latitude': 43.669542,
 'Neighborhood Longitude': -79.4225637}

### Third Best Neighbourhood Details

In [47]:
neigh_summary[neigh_summary['Group'] == 2]

,Neighborhood,Group
0,Berczy Park,2
5,"Commerce Court, Victoria Hotel",2
7,"Garden District, Ryerson",2
11,"Regent Park, Harbourfront",2
14,St. James Town,2
15,"St. James Town, Cabbagetown",2
16,Stn A PO Boxes,2
17,"Toronto Dominion Centre, Design Exchange",2


In [53]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 2]['Neighborhood'])[0]
toronto_venues[toronto_venues['Neighbourhood'] == name_of_neigh].iloc[0,1:4].to_dict()

{'Neighbourhood': 'Berczy Park',
 'Neighborhood Latitude': 43.6447708,
 'Neighborhood Longitude': -79.3733064}